# Extracción de características

La extracción de características consiste en extraer toda la información posible y relevante de las reviews de cada estación. Esto nos permitirá dar recomendaciones de las rutas más personalizadas al poder aplicar filtros con la información que consigamos obtener a través de las reviews.

## Imports y descargas necesarios

In [1]:
import os
import csv
import pickle
import nltk
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
import spacy
from collections import defaultdict, Counter
import stanza
from deep_translator import GoogleTranslator
from nltk.corpus import wordnet as wn
from unidecode import unidecode
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\evano\Documents\GitHub\TFG\tfg-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nltk.download('punkt_tab')
nltk.download("stopwords")
nltk.download('wordnet')
stanza.download("es")  # Descargar el modelo en español

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\evano\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\evano\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\evano\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2025-05-01 11:05:28 INFO: Downloaded file to C:\Users\evano\stanza_resources\resources.json
2025-05-01 11:05:28 INFO: Downloading default packages for language: es (Spanish) ...
2025-05-01 11:05:29 INFO: File exists: C:\Users\evano\stanza_resources\es\default.zip
2025-05-01 11:05:35 INFO: Finished downloading models and saved to C:\Users\evano\stanza_resources


## Implementación de métodos para extracción de características

### N-gramas

Para la extracción de características vamos a emplear n-gramas. Un n-grama es una secuencia de n palabras consecutivas en un texto. Esto nos va a permitir poder sacar la infomación relevante de las reseñas al poder aplicar distintos n-gramas y ver sus relaciones.

Vamos primero a probar con unas reseñas de prueba para ver como de bien funcionan los n-gramas:

In [3]:
reviews = [
    "La estación de metro es moderna y limpia, pero los trenes son lentos.",
    "A veces hay demasiada gente y el servicio es malo.",
    "El metro de Madrid es rápido y eficiente, aunque algunas estaciones están sucias.",
    "Buena conexión con otras líneas, pero los horarios nunca son confiables."
]

Para asegurarnos de conseguir la información más relevante vamos a eliminar de nuestras reviews de prueba todas las stopwords

Eliminamos las stopwords en español de las reviews de prueba

In [4]:
#Obtenemos las stopwords en español
spanishStopwords = list(stopwords.words("spanish"))

# Eliminar stopwords de cada review
filteredReviews = [
    " ".join([word for word in word_tokenize(review.lower()) if word not in spanishStopwords and word.isalnum()])
    for review in reviews
]

# Imprimir resultado
print(filteredReviews)

['estación metro moderna limpia trenes lentos', 'veces demasiada gente servicio malo', 'metro madrid rápido eficiente aunque estaciones sucias', 'buena conexión líneas horarios nunca confiables']


Una vez eliminadas las stopwords procedemos a usar los n-gramas para poder analizarlas y extraer las características. 
Para que sea lo más fiable posible vamos a usar n-gramas de 2 a 5, es decir, desde bigramas hasta pentagramas, para obtener una precisión que sea la mejor posible. Si solo tenemos bigramas nos vamos a dejar información relevante pero si usamos n-gramas muy grandes la precisión también se vería reducida, por lo que se optó por usar n-gramas de 2 a 5.

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

# Crear un vectorizador de ngramas
vectorizer = CountVectorizer(ngram_range=(2, 5))

# Aplicar a las reseñas
X = vectorizer.fit_transform(filteredReviews)

# Ver n-gramas encontrados
ngramas = vectorizer.get_feature_names_out()
print("ngramas encontrados:", ngramas)

# Ver la matriz de frecuencia
print("Matriz de ngramas:\n", X.toarray())

ngramas encontrados: ['aunque estaciones' 'aunque estaciones sucias' 'buena conexión'
 'buena conexión líneas' 'buena conexión líneas horarios'
 'buena conexión líneas horarios nunca' 'conexión líneas'
 'conexión líneas horarios' 'conexión líneas horarios nunca'
 'conexión líneas horarios nunca confiables' 'demasiada gente'
 'demasiada gente servicio' 'demasiada gente servicio malo'
 'eficiente aunque' 'eficiente aunque estaciones'
 'eficiente aunque estaciones sucias' 'estaciones sucias' 'estación metro'
 'estación metro moderna' 'estación metro moderna limpia'
 'estación metro moderna limpia trenes' 'gente servicio'
 'gente servicio malo' 'horarios nunca' 'horarios nunca confiables'
 'limpia trenes' 'limpia trenes lentos' 'líneas horarios'
 'líneas horarios nunca' 'líneas horarios nunca confiables'
 'madrid rápido' 'madrid rápido eficiente'
 'madrid rápido eficiente aunque'
 'madrid rápido eficiente aunque estaciones' 'metro madrid'
 'metro madrid rápido' 'metro madrid rápido eficien

Una vez obtenidos los n-gramas de las reviews filtradas sin los stopwords lo que vamos a hacer es quedarnos de cada n-grama con el sustantivo y su adjetivo adyacente más cercano. Esto es debido a que en español los adjetivos calificativos suelen ir al lado o muy cerca del sustantivo al que califica. De esta manera podremos sacar todas las características.

Nos descargamos e importamos las librerías necesarias para ello. En este caso, hacemos uso de la librería spaCy de python para poder saber la categoría gramatical de las palabras en español:

In [6]:
nlp = spacy.load("es_core_news_sm")

In [7]:
# Crear el mapa: {sustantivo: {(adjetivo))}}
caracteristicasNGramas = defaultdict(set)

for ngrama in ngramas:
    doc = nlp(ngrama)
    sustantivoActual = None
    for token in doc:
        if token.pos_ == "NOUN":
            sustantivoActual = token.lemma_.lower()
        elif token.pos_ == "ADJ" and sustantivoActual:
            caracteristicasNGramas[sustantivoActual].add(token.lemma_.lower())

print(caracteristicasNGramas)

defaultdict(<class 'set'>, {'estación': {'sucio'}, 'conexión': {'línea', 'confiable', 'horario'}, 'gente': {'malo', 'servicio'}, 'servicio': {'malo'}, 'moderna': {'limpio'}, 'horario': {'confiable'}, 'tren': {'lento'}, 'línea': {'confiable', 'horario'}, 'metro': {'rápido', 'moderno', 'eficiente'}})


Se puede apreciar como para las reviews de ejemplo que hemos utilizado hay incongruencias, es decir, se registra que los trenes son lentos pero que el metro es rápido. Además, la librería interpreta algunas palabras como sustantivos o como adjetivos que no son.

### Librerías con redes neuronales

Con los n-gramas solo puedes ver las palabras que tengan cerca. Esto en la mayoria de casos es lo que buscas y necesitas pero sin embargo en otras ocasiones un adjetivo al final de la oración puede referirse a un sustantivo del principio o un adjetivo cerca de un sustantivo puede referirse a otro sustantivo, y esto con los n-gramas no se puede ver. Para ello, haremos uso de las distinntas librerías de python para realizar un análisis más exahustivo de cada review y poder tener resultados más precisos.

El uso de estas librerías va a ser muy similar al de los n-gramas. Vamos a sacar los distintos adjetivos que esten relacionados con los sustantivos correspondientes. De esta manera se ahorra tiempo ya que podemos hacerlo evitando la repetición de los n-gramas.

#### SpaCy

La primera libreía que vamos a utilizar es la de spaCy, previamente utilizada para saber que clase de palabra era cada una en las reviews. Sin embargo, esta librería no solo es capaz de saber la clase gramatical de las palabras sino que además es capaz de decirte con que palabra esta relacionada, que es justo lo que necesitamos saber para extraer las características de las distintas reviews.

In [8]:
reviews = [
    "La estación de metro es moderna y limpia, pero los trenes son lentos.",
    "A veces hay demasiada gente y el servicio es malo.",
    "El metro de Madrid es rápido y eficiente, aunque algunas estaciones están sucias.",
    "Buena conexión con otras líneas, pero los horarios nunca son confiables.",
    "La estacion es muy fea y está muy mal cuidada",
    "Los horarios son muy utiles"
]

In [9]:
nlp = spacy.load("es_core_news_md")

In [10]:
# Treat both common nouns (NOUN) and proper nouns (PROPN) as valid targets
NOUN_POS = {"NOUN", "PROPN"}

def collectForms(adj):
    """Return every adjectival *surface form* that belongs to `adj`.

    Includes:
    * the base lemma itself
    * any preceding adverb modifiers (advmod) in document order
    * any coordinated adjectives (dep == "conj") that have NO own subject
    """
    # Adverbial intensifiers/modifiers to the adjective
    advs = sorted(
        [c for c in adj.children if c.dep_ == "advmod" and c.pos_ == "ADV"],
        key=lambda c: c.i
    )
    adv_lemmas = [c.lemma_.lower() for c in advs]
    base = adj.lemma_.lower()

    forms = set()
    forms.add(" ".join(adv_lemmas + [base]) if adv_lemmas else base)

    # Recursively walk into adjective conjuncts
    for child in adj.children:
        if child.dep_ == "conj" and child.pos_ == "ADJ":
            # skip conjuncts that have their own subject (they describe
            # a different entity)
            if not any(c.dep_ == "nsubj" and c.pos_ in NOUN_POS
                       for c in child.children):
                forms |= collectForms(child)
    return forms

def findNoun(adj):
    """Given an ADJ (or participle), locate the noun it describes.

    Handles:
    a) 'amod' modifiers
    b) copular/adjectival complements ('acomp' / 'attr')
    c) coordinated adjectives
    d) participial relative clauses ('acl')
    e) adjectives that carry their own nsubj
    f) adjectives that are the sentence ROOT ("Los trenes son LENTOS")
    g) fallback: climb ancestors until a NOUN/PROPN is found
    """
    # a) direct pre-nominal modifier
    if adj.dep_ == "amod" and adj.head.pos_ in NOUN_POS:
        return adj.head

    # b) copular predicate after SER/ESTAR
    if adj.dep_ in {"acomp", "attr"} and adj.head.pos_ in {"VERB", "AUX"}:
        for child in adj.head.children:
            if child.dep_ == "nsubj" and child.pos_ in NOUN_POS:
                return child

    # c) coordinated adjective – inherit target from the head adjective
    if adj.dep_ == "conj" and adj.head.pos_ == "ADJ":
        return findNoun(adj.head)

    # d) participle used as adjectival clause
    if adj.dep_ == "acl" and adj.head.pos_ in NOUN_POS:
        return adj.head

    # e) the adjective has its own nominal subject
    for child in adj.children:
        if child.dep_ == "nsubj" and child.pos_ in NOUN_POS:
            return child

    # f) adjective is ROOT; subject may sit under the AUX
    if adj.dep_ == "ROOT":
        # direct subject
        for child in adj.children:
            if child.dep_ == "nsubj" and child.pos_ in NOUN_POS:
                return child
        
            if child.pos_ in {"AUX", "VERB"}:
                for gc in child.children:
                    if gc.dep_ == "nsubj" and gc.pos_ in NOUN_POS:
                        return gc

    # g) climb ancestors as last resort
    up = adj
    while up.dep_ != "ROOT":
        up = up.head
        if up.pos_ in NOUN_POS:
            return up

    return None

def extractAspects(reviews):
    """Main entry: return defaultdict(Counter)."""
    nlp = spacy.load("es_core_news_sm")
    aspects = defaultdict(set)

    fallbackLlema = "estacion" # If no noun is found this would be the default

    for doc in nlp.pipe(reviews, batch_size=20):
        for token in doc:
            # accept adjectives and adjectival participles
            if not (token.pos_ == "ADJ" or "Part" in token.morph.get("VerbForm") or (token.dep_ == "ROOT" and token.pos_ in {"INTJ", "ADV", "PROPN"})):
                continue
            # skip conjunct duplicates (will be handled via their head)
            if token.dep_ == "conj" and token.head.pos_ == "ADJ" and token.pos_ == "ADJ":
                continue

            noun = findNoun(token)

            if noun is None:
                if token.dep_ == "ROOT":
                        noun_key = fallbackLlema
                else:
                        continue
            else:
                noun_key = unidecode(noun.lemma_.lower()) 
                
            for form in collectForms(token):
                if (form == noun_key or unidecode(form) == noun_key):
                   continue 
                
                aspects[noun_key].add(form)   

    return aspects


In [11]:
caracteristicasSpacy = extractAspects(reviews)

In [12]:
caracteristicasSpacy

defaultdict(set,
            {'estacion': {'limpio',
              'mal cuidado',
              'moderno',
              'mucho feo',
              'sucio'},
             'servicio': {'malo'},
             'metro': {'eficiente', 'rápido'},
             'conexion': {'buen'},
             'horario': {'mucho util', 'nunca confiable'}})

#### Stanza

La otra librería que vamos a utilizar es stanza que es una librería de python desarrollada por Standford. Esta librería es muy similar a la de spaCy, la cual, también permite saber la clase gramatical a la cual pertenece la palabra y con que palabra en la oración esta relacionada. 

In [13]:
nlp = stanza.Pipeline("es", processors="tokenize,mwt,pos,lemma,ner,depparse")

2025-04-29 16:15:30 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-04-29 16:15:30 INFO: Downloaded file to C:\Users\evano\stanza_resources\resources.json
2025-04-29 16:15:31 INFO: Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |
| ner       | conll02           |

2025-04-29 16:15:31 INFO: Using device: cpu
2025-04-29 16:15:31 INFO: Loading: tokenize
2025-04-29 16:15:35 INFO: Loading: mwt
2025-04-29 16:15:35 INFO: Loading: pos
2025-04-29 16:15:38 INFO: Loading: lemma
2025-04-29 16:15:40 INFO: Loading: depparse
2025-04-29 16:15:41 INFO: Loading: ner
2025-04-29 16:15:44 INFO: Done loading processors!


##### Procesar cada review

In [14]:
def processReviewsStanza(reviews):

    caracteristicasStanza = defaultdict(set)
    
    for review in reviews:
        doc = nlp(review)
        for sentence in doc.sentences:
            for word in sentence.words:
                if word.upos == "NOUN":
                    sustantivo = word.lemma.lower()
                    headWord = sentence.words[word.head - 1]
                    if headWord.upos == "ADJ":
                        adjetivo = headWord.lemma.lower()
                        caracteristicasStanza[sustantivo].add(adjetivo)
                elif word.upos == "ADJ":
                    adjetivo = word.lemma.lower()
                    headWord = sentence.words[word.head - 1]
                    if headWord.upos == "NOUN":
                        sustantivo = headWord.lemma.lower()
                        caracteristicasStanza[sustantivo].add(adjetivo)

    return caracteristicasStanza

In [15]:

caracteristicasStanza = processReviewsStanza(reviews)

### Comparación de resultados

Vamos a ver que resultados sacan cada opción previamente explicadas y compararlas entre sí. De esta manera nos quedaremos con el que mejor resultados de para utilizarlo con todas y cada una de las reviews de las distintas estaciones de metro que tenemos.

In [16]:
print("Las caracteristicas de los n-gramas \n")
print(caracteristicasNGramas)
print("\n")
print("Las caracteristicas de spacy \n")
print(caracteristicasSpacy)
print("\n")
print("Las caracteristicas de stanza \n")
print(caracteristicasStanza)

Las caracteristicas de los n-gramas 

defaultdict(<class 'set'>, {'estación': {'sucio'}, 'conexión': {'línea', 'confiable', 'horario'}, 'gente': {'malo', 'servicio'}, 'servicio': {'malo'}, 'moderna': {'limpio'}, 'horario': {'confiable'}, 'tren': {'lento'}, 'línea': {'confiable', 'horario'}, 'metro': {'rápido', 'moderno', 'eficiente'}})


Las caracteristicas de spacy 

defaultdict(<class 'set'>, {'estacion': {'limpio', 'mucho feo', 'mal cuidado', 'moderno', 'sucio'}, 'servicio': {'malo'}, 'metro': {'rápido', 'eficiente'}, 'conexion': {'buen'}, 'horario': {'mucho util', 'nunca confiable'}})


Las caracteristicas de stanza 

defaultdict(<class 'set'>, {'estación': {'sucia', 'moderno'}, 'tren': {'lento'}, 'servicio': {'malo'}, 'metro': {'rápido'}, 'conexión': {'buena', 'confiable'}, 'horario': {'util', 'confiable'}, 'estacion': {'feo'}})


Podemos observar como con los n-gramas parece que se sacan más características que con las librerías. Sin embargo, si lo analizamos más profundamente podemos ver como con los n-gramas se añaden palabras que no son sustantivos o que no son adjetivos o que no tienen ninguna relevancia. Esto demuestra que los resultados con las librerías son más precisas. Sin embargo, las librerías alguna característica como que la estación esta limpia también se la ha saltado. 

Entre las dos librerías usadas vemos como dan resultados exactamente iguales, por lo que nos quedaremos con spacy debido a que podemos ver los hijos de una palabra y la palabra con la que esta relacionada. Esto cuando haya más reviews y más largas da más seguridad en que se van a sacar el máximo de características posibles aunque se pierdan algunas como ya hemos podido ver anteriormente. 

Por todo esto la librería que vamos a usar para extraer todas las psoibles características va a ser spacy.

## Obtención de todas las características de las reviews

Una vez realizada la comparación nos quedaremos con la librería spacy que es la que mejor resultados obtuvo. A continuación realizamos la extracción de las caracteristicas para cada una de las distintas estaciones de metro.

Para ello vamos a realizar otra pasada a las reviews y contando su frecuencia para después ver si son relevantes o no

Función auxiliar para contar la frecuencia:

In [17]:
NOUN_POS = {"NOUN", "PROPN"}

def collectForms(adj):
    """Return every adjectival *surface form* that belongs to `adj`.

    Includes:
    * the base lemma itself
    * any preceding adverb modifiers (advmod) in document order
    * any coordinated adjectives (dep == "conj") that have NO own subject
    """
    # Adverbial intensifiers/modifiers to the adjective
    advs = sorted(
        [c for c in adj.children if c.dep_ == "advmod" and c.pos_ == "ADV"],
        key=lambda c: c.i
    )
    adv_lemmas = [c.lemma_.lower() for c in advs]
    base = adj.lemma_.lower()

    forms = set()
    forms.add(" ".join(adv_lemmas + [base]) if adv_lemmas else base)

    # Recursively walk into adjective conjuncts
    for child in adj.children:
        if child.dep_ == "conj" and child.pos_ == "ADJ":
            # skip conjuncts that have their own subject (they describe
            # a different entity)
            if not any(c.dep_ == "nsubj" and c.pos_ in NOUN_POS
                       for c in child.children):
                forms |= collectForms(child)
    return forms

def findNoun(adj):
    """Given an ADJ (or participle), locate the noun it describes.

    Handles:
    a) 'amod' modifiers
    b) copular/adjectival complements ('acomp' / 'attr')
    c) coordinated adjectives
    d) participial relative clauses ('acl')
    e) adjectives that carry their own nsubj
    f) adjectives that are the sentence ROOT ("Los trenes son LENTOS")
    g) fallback: climb ancestors until a NOUN/PROPN is found
    """
    # a) direct pre-nominal modifier
    if adj.dep_ == "amod" and adj.head.pos_ in NOUN_POS:
        return adj.head

    # b) copular predicate after SER/ESTAR
    if adj.dep_ in {"acomp", "attr"} and adj.head.pos_ in {"VERB", "AUX"}:
        for child in adj.head.children:
            if child.dep_ == "nsubj" and child.pos_ in NOUN_POS:
                return child

    # c) coordinated adjective – inherit target from the head adjective
    if adj.dep_ == "conj" and adj.head.pos_ == "ADJ":
        return findNoun(adj.head)

    # d) participle used as adjectival clause
    if adj.dep_ == "acl" and adj.head.pos_ in NOUN_POS:
        return adj.head

    # e) the adjective has its own nominal subject
    for child in adj.children:
        if child.dep_ == "nsubj" and child.pos_ in NOUN_POS:
            return child

    # f) adjective is ROOT; subject may sit under the AUX
    if adj.dep_ == "ROOT":
        # direct subject
        for child in adj.children:
            if child.dep_ == "nsubj" and child.pos_ in NOUN_POS:
                return child
        
            if child.pos_ in {"AUX", "VERB"}:
                for gc in child.children:
                    if gc.dep_ == "nsubj" and gc.pos_ in NOUN_POS:
                        return gc

    # g) climb ancestors as last resort
    up = adj
    while up.dep_ != "ROOT":
        up = up.head
        if up.pos_ in NOUN_POS:
            return up

    return None

def extractAspects(reviews):
    """Main entry: return defaultdict(Counter)."""
    nlp = spacy.load("es_core_news_sm")
    aspects = defaultdict(Counter)

    fallbackLlema = "estacion" # If no noun is found this would be the default

    for doc in nlp.pipe(reviews, batch_size=20):
        for token in doc:
            # accept adjectives and adjectival participles
            if not (token.pos_ == "ADJ" or "Part" in token.morph.get("VerbForm") or (token.dep_ == "ROOT" and token.pos_ in {"INTJ", "ADV", "PROPN"})):
                continue
            # skip conjunct duplicates (will be handled via their head)
            if token.dep_ == "conj" and token.head.pos_ == "ADJ" and token.pos_ == "ADJ":
                continue

            noun = findNoun(token)

            if noun is None:
                if token.dep_ == "ROOT":
                        noun_key = fallbackLlema
                else:
                        continue
            else:
                noun_key = unidecode(noun.lemma_.lower()) 
                
            for form in collectForms(token):
                if (form == noun_key or unidecode(form) == noun_key):
                   continue 
                
                aspects[noun_key][form] += 1

    return aspects

In [18]:
reviews = [
    "La estación de metro es moderna y limpia, pero los trenes son lentos.",
    "A veces hay demasiada gente y el servicio es malo.",
    "El metro de Madrid es rápido y eficiente, aunque algunas estaciones están sucias.",
    "Buena conexión con otras líneas, pero los horarios nunca son confiables.",
    "Excelente",
    "casa bonita. Horrible"
]

In [19]:
results = extractAspects(reviews)

In [20]:
results

defaultdict(collections.Counter,
            {'estacion': Counter({'moderno': 1,
                      'limpio': 1,
                      'sucio': 1,
                      'excelente': 1,
                      'horrible': 1}),
             'servicio': Counter({'malo': 1}),
             'metro': Counter({'rápido': 1, 'eficiente': 1}),
             'conexion': Counter({'buen': 1}),
             'horario': Counter({'nunca confiable': 1}),
             'casa': Counter({'bonito': 1})})

Función para cargar todas las reviews de un csv y devolverlas en una lista

In [21]:
def loadReviews(path):
    with open(path, encoding="utf-8", newline="") as f:
        reader = csv.reader(f, quotechar='"', skipinitialspace=True)
        rows = [row[0] for row in reader if row]

    if rows and rows[0].strip().lower() == "review":
        rows = rows[1:]

    return rows

Función para guardar el objeto devulelto por extractAspects (pensado para guardar el objeto de python al completo)

In [22]:
def saveCharacteristics(path, data):
    with open(path, "wb") as file:
        pickle.dump(data, file)

Función para guardar la información devuelta por extractAspects en un .txt

In [ ]:
def saveCharacteristicsAsTxt(path, dataCounter):
    with open(path, "w", encoding = "utf-8") as file:
        for key in dataCounter:
            file.write(str(key) + ": ")

            for subkey in dataCounter[key]:
                file.write("(" + str(subkey) + ", " + str(dataCounter[key][subkey]) + ")" )
            
            file.write("\n")

In [24]:
inputFolderPath = "../1. Data/5. Classified Reviews"
outputFolderPath = "../1. Data/6. Reviews Characteristics/1. Raw"

for station in os.listdir(inputFolderPath):

    if not station.lower().endswith(".csv"):
        continue

    # Import the data
    inputPath = os.path.join(inputFolderPath, station)
    reviews = loadReviews(inputPath)

    # Process the data
    results = extractAspects(reviews)

    # Save the data
    if station.endswith('.csv'):
        station = station[:-4]

    outputPathPkl = os.path.join(outputFolderPath, station + ".pkl")
    outputPathTxt = os.path.join(outputFolderPath, station + ".txt")

    saveCharacteristics(outputPathPkl, results)
    saveCharacteristicsAsTxt(outputPathTxt, results)

## Análisis de la extracción

Función para cargar las reviews

In [25]:
def loadReviews(path):
    with open(path, encoding="utf-8", newline="") as f:
        reader = csv.reader(f, quotechar='"', skipinitialspace=True)
        rows = [row[0] for row in reader if row]

    if rows and rows[0].strip().lower() == "review":
        rows = rows[1:]

    return rows

Función para cargar las características desde los ficheros

In [26]:
def loadCharacteristics(path):
    with open(path, "rb") as f:
        return pickle.load(f)

Se van a escoger tres estaciones al azar y se va a analizar la calidad de la extracción de características

In [27]:
inputFolderPath = "../1. Data/6. Reviews Characteristics/1. Raw"

### Abrantes

#### Reviews

In [28]:
inputPath = "../1. Data/5. Classified Reviews/Abrantes.csv"
reviews = loadReviews(inputPath)

In [29]:
reviews

['Tiene fácil acceso para las personas con movilidad reducida, una salida hacia el colegio Julián Besteiro y el otro en frente del Burgen King.',
 'Estación de metro que pertenece a la línea 11. No tiene cobertura móvil como en toda la linea. Dispone de ascensor. Tiene 2 accesos a la calle. La estación da un poco de mal rollo porque no es muy transitada.',
 'Es una estación muy tranquila, ya que sólo circula por ella la línea de metro 11 de La Fortuna a Plaza Elíptica.',
 'Los trabajadores de metro de esta estación son muy amargados... Te dicen las cosas de muy mala gana...',
 'Asqueroso',
 'Esta muy bien ubicado',
 'Estacion con mal olor',
 'Espero que hayan mejorais',
 'Excelente',
 'Excelente']

#### Extracción de características

In [30]:
characteristics = loadCharacteristics(os.path.join(inputFolderPath, "Abrantes.pkl"))

In [31]:
characteristics

defaultdict(collections.Counter,
            {'acceso': Counter({'fácil': 1}),
             'movilidad': Counter({'reducido': 1}),
             'cobertura': Counter({'móvil': 1}),
             'rollo': Counter({'mal': 1}),
             'estacion': Counter({'excelente': 2,
                      'mucho tranquilo': 1,
                      'asqueroso': 1}),
             'trabajador': Counter({'mucho amargado': 1}),
             'olor': Counter({'mal': 1})})

Como se puede ver, la extracción de características es bastante certera

## Detección de sinónimos

In [3]:
# Cargar modelo multilingüe
model1 = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
model2 = SentenceTransformer("sentence-transformers/LaBSE")

def areSynonyms(palabra1, palabra2, umbral=0.65):
    emb1 = model1.encode([palabra1])
    emb2 = model1.encode([palabra2])
    emb11 = model2.encode([palabra1])
    emb22 = model2.encode([palabra2])
    sim1 = cosine_similarity(emb1, emb2)[0][0]
    sim2 = cosine_similarity(emb11, emb22)[0][0]
    sim = ((sim1 + sim2) / 2)
    return  sim >= umbral

## Detección de antónimos

Como puede haber discrepancias entre las caracteristicas que hemos sacado debido a la variabilidad de todas las reviews solo se van a quedar los que no tengan antonimos para un mismo sustantivo o el antonimo que mayor frecuencia tenga. Por ejemplo, si tenemos que el metro esta limpio con una frecuencia de 7 y que el metro esta sucio con una frecuencia de 5, se quedaría que el metro esta limpio.

Sin embargo, para poder sacar los antonimos de la mejor manera posible vamos a traducirlos al ingles y luego de vuelta al español debido a que en ingles esta mucho mejor optimizado y falla mucho menos que en español.

In [4]:
# Traducir palabra al ingles
def traducir(palabra, sr, tg):
    traduccion = GoogleTranslator(source=sr, target=tg).translate(palabra)
    return traduccion

In [5]:
# Obtener todos los antonimos
def getAntonimos(palabra):
    antonimos = set()
    for sentido in wn.synsets(palabra, pos=wn.ADJ):  # todos los significados de la palabra como adjetivo
        for lema in sentido.lemmas(): # cada sinónimo dentro de ese significado
            for antonimo in lema.antonyms():  # para cada sinónimo, se busca sus antónimos
                antonimos.add(antonimo.name()) # se añade el nombre del antónimo al set
    return antonimos

In [6]:
ant = getAntonimos(traducir("pequeño", "es", "en"))

newAnt = []
for elem in ant:
    newAnt.append(traducir(elem, "en", "es"))
newAnt

['mucho', 'grande', 'alto', 'grande']

## Funciones para procesar un conjunto de características de una estación:

### 1. Eliminación de sustantivos que sean sinónimos

In [7]:
def combineKeys(d):
    
    """
    Input:
    d : A Counter where each key is a noun,
              and each value is another Counter mapping adjetives and each frequency.
              
    Group the Counters keys that are synonyms
    
    Output:
    final: A new Counter with synonymous keys merged into a single entry,
          summing their Counter values.
    """
    
    final = {} # New dictionary to hold merged entries
    processed = set() # keys that have already been processed
    keys = list(d.keys()) # List of original keys

    for i, key in enumerate(keys):
        if key in processed:
            continue # Skip if this key has already been processed as a synonym

        total = Counter(d[key]) # Counter of the current key

        for j in range(i + 1, len(keys)):
            k2 = keys[j] # New key to see if its a synonym
            if k2 in processed:
                continue # Skip if this key has already been processed as a synonym
            if areSynonyms(key, k2): # If the two keys are synonyms
                total += d[k2] # Add frequencies from k2 into total
                processed.add(k2) # Mark k2 as processed

        final[key] = total # Save the merged result under the original key
        processed.add(key) # Mark the current key as processed

    return Counter(final) 


### 2. Combinación de sinónimos en las características

In [8]:
def combineSynonyms(d):

    """
    Input:
    d : A Counter where each key is a noun,
              and each value is another Counter mapping adjetives and each frequency.

    Groups synonymous adjectives (features) for each noun key in the dictionary.
    For each group of synonyms, it keeps the one with the most antonyms as the representative.

    Output:
    final: A new Counter with the same noun keys,
          but with synonymous adjectives grouped and frequencies combined.
    """

    final = {} # New dictionary to hold the merged synonymous adjectives and frequencies

    for noun in d.keys():
        processed = set() # Adjetives that have already been processed
        features = list(d[noun].keys()) # List of all adjectives for this noun
        newFeatures = {} # New grouped features for this noun

        for idx, feature in enumerate(features):

            rep = feature # Representative feature
            numAntRep = len(getAntonimos(feature)) # Number of antonyms for current representative

            if feature in processed:
                continue # Skip if this adjective was already merged

            total = d[noun][feature] # Start with the frequency of the current feature

            for j in range(idx + 1, len(features)):

                feature2 = features[j] # New feature to see if its a synonym

                if feature2 in processed:
                    continue # Skip if already processed

                if areSynonyms(feature, feature2): # If the two features are synonyms
                    total += d[noun][feature2] # Add the frequency
                    processed.add(feature2) # Mark as processed
                    
                    numAntFeature2 = len(getAntonimos(feature2)) # Get number of new feature antonyms
                    if numAntFeature2 > numAntRep: # Compare number of antonyms
                        numAntRep = numAntFeature2
                        rep = feature2  # Use the feature with more antonyms as representative

            
            newFeatures.update({rep: total}) # Add the merged features 
            processed.add(feature) # Mark the current feature as processed
        
        final[noun] = Counter(newFeatures) # Save the merged features to its noun

    return Counter(final)
        

### 3. Filtrado de las características que sean antónimos

In [9]:
globalAntonyms = {}

def removeAntonyms(d):

    """
    Input:
    d : A Counter where each key is a noun,
              and each value is another Counter mapping adjetives and each frequency.

    For each noun, adjective pairs that are antonyms are removed.    
    If two adjectives are antonyms, only the one with the highest frequency is kept.

    Output:
    final: A new Counter with the same noun keys,
          but with synonymous adjectives grouped and frequencies combined.
    """
    final = {} # Counter to store filtered adjectives per noun

    for noun, adjs in d.items():
        lst = list(adjs) # Convert the adjectives to a list for indexed access
        newAdjs = {} # Store final adjectives after removing antonyms
        processed = set() # Adjetives that have already been processed

        for i, adj in enumerate(lst):
            if adj in processed:
                continue # Skip already processed adjectives

            if adj not in globalAntonyms:
                antonimosIngles = getAntonimos(traducir(adj,'es','en')) # Translate the adjective to English and get its antonyms
                antonimos = [traducir(ant, 'en', 'es').lower() for ant in antonimosIngles] # Translate each antonym back to Spanish
                globalAntonyms[adj] = antonimos

            else:
                antonimos = globalAntonyms[adj]

            emparejado = False # Flag to indicate if an antonym match was found

            for j in range(i + 1, len(lst)):
                adj2 = lst[j]
                if adj2 in processed: 
                    continue # Skip already processed adjectives

                if adj2 in antonimos: # If adj2 is an antonym of adj
                    if d[noun][adj] >= d[noun][adj2]: # Keep the adjective with the higher frequency
                        newAdjs.update({adj: d[noun][adj]})
                    else:
                        newAdjs.update({adj2: d[noun][adj2]})

                    processed.update([adj, adj2]) # Mark both as processed 
                    emparejado = True # Antonym match was found
                    break # Break to avoid duplicate grouping

            if not emparejado: # If no antonym was found for adj, keep it as it was
                newAdjs.update({adj: d[noun][adj]})
                processed.add(adj) # Mark adj as processed 

        final[noun] = Counter(newAdjs) # Assign the final filtered adjective Counter to the noun
    
    return Counter(final)

## Procesamiento de las características de todas las estaciones

Función para cargar las características de una estación

In [10]:
def loadCharacteristics(path):
    with open(path, "rb") as f:
        return pickle.load(f)

Función para guardar las características de una estación

In [11]:
def saveCharacteristics(path, data):
    with open(path, "wb") as file:
        pickle.dump(data, file)

Función para guardar las características de una estación en un .txt

In [12]:
def saveCharacteristicsAsTxt(path, dataCounter):
    with open(path, "w", encoding = "utf-8") as file:
        for key in dataCounter:
            file.write(str(key) + ": ")

            for subkey in dataCounter[key]:
                file.write("(" + str(subkey) + ", " + str(dataCounter[key][subkey]) + ")" )
            
            file.write("\n")

In [13]:
inputFolderPath = "../1. Data/6. Reviews Characteristics/1. Raw"
outputFolderPath = "../1. Data/6. Reviews Characteristics/2. Filtered"

# Dictionary with the characteristics of all the stations
data = {}

adjectives = {}

for station in os.listdir(inputFolderPath):

    if not station.lower().endswith(".pkl"):
        continue

    # Import the data
    inputPath = os.path.join(inputFolderPath, station)
    characteristics = loadCharacteristics(inputPath)

    # Combine synonyms from the keys (nouns)
    finalCharacteristics = combineKeys(characteristics)

    # Combine synonyms from the adjectives
    finalCharacteristics = combineSynonyms(finalCharacteristics)

    # Remove antonyms
    finalCharacteristics = removeAntonyms(finalCharacteristics)

    # Add it to the dictionary with all the information
    data[station[:-4]] = finalCharacteristics

    # Save the processed characteristics
    outputPath = os.path.join(outputFolderPath, station[:-4])

    saveCharacteristics(outputPath + ".pkl", finalCharacteristics)
    saveCharacteristicsAsTxt(outputPath + ".txt", finalCharacteristics)


    for noun in finalCharacteristics.keys():

        for adj in finalCharacteristics[noun].keys():

            if adj in adjectives:
                adjectives[adj] += finalCharacteristics[noun][adj]
            else:
                adjectives[adj] = finalCharacteristics[noun][adj]


outputPath = os.path.join(outputFolderPath, "totalAdjectives")
saveCharacteristics(outputPath + ".pkl", finalCharacteristics)

# Save the info as a txt
with open(outputPath + ".txt", "w", encoding = "utf-8") as file:
    for ajd in adjectives.keys():
        file.write(adj + ": " +  str(adjectives[adj]) + "\n")


AttributeError: 'NoneType' object has no attribute 'lower'

Una vez hemos quitado los antonimos con menos frecuencia, es decir, caracteristicas irrelevantes, vamos a añadir y guardar las características sin la frecuencia.

In [ ]:
caracteristicasFinal = {}

def quitarFrecuencias(dicF, dic):
    for sustantivo, adjetivos in dicF.items():
        dic[sustantivo] = {adj for adj, _ in adjetivos}
    return dic

caracteristicasFinal = quitarFrecuencias(aux, caracteristicasFinal)
print(caracteristicasFinal)

{'estación': {'sucio', 'moderno'}, 'tren': {'lento'}, 'servicio': {'malo'}, 'metro': {'limpio', 'rápido'}, 'conexión': {'buen', 'confiable'}, 'horario': {'confiable'}}
